In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import folium
import numpy as np
import seaborn as sns
import os
import fastparquet
import warnings
import geopy
from geopy.point import Point
import time
warnings.simplefilter(action='ignore', category=FutureWarning)

#df = pd.read_parquet('Downloads/airbnb_parquets')
#df.to_csv('csv_printout.csv')

In [2]:
def append_fig_to_html(list_of_figs):
    for fig in list_of_figs:
        with open("reports/report_draft.html",'a') as f:
            f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

# write this later
"""
def overwrite_html(list_of_figs):
    
    with open("reports/report_draft.html",'w') as f:
    f.write(listing_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(price_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(price_v_numlist_fig.to_html(full_html=False, include_plotlyjs='cdn'))
"""

def calculate_mortgage(home_value, interest_rate, num_years, down_payment_pct=0):
    per_payment_interest = 0
    loan_value = 0

    if down_payment_pct >= 1:
        down_payment = down_payment_pct/100 * home_value
        loan_value = home_value - down_payment
    else:
        down_payment = down_payment_pct * home_value
        loan_value = home_value - down_payment

    if loan_value/home_value < 0.80: 
        # insert pmi calc here
        pmi = 0.01  # using near average value here
        pmi_cost = 0.0007*home_value    # shot in the dark after interpolating nerdwallet calculator
        if interest_rate >= 1:
            per_payment_interest = interest_rate/100/12
        else:
            per_payment_interest = interest_rate/12
        num_months = num_years*12
        mortgage = loan_value*(per_payment_interest*(1+per_payment_interest)**num_months)/((1+per_payment_interest)**num_months-1) + pmi_cost

        mortgage = np.round(mortgage, 2)
        return mortgage
    else:
        if interest_rate >= 1:
            per_payment_interest = interest_rate/100/12
        else:
            per_payment_interest = interest_rate/12
        num_months = num_years*12
        mortgage = loan_value*(per_payment_interest*(1+per_payment_interest)**num_months)/((1+per_payment_interest)**num_months-1)

        mortgage = np.round(mortgage, 2)
        return mortgage

def calculate_roi(airbnb_daily_price, occupancy_rate, monthly_mortgage, monthly_maintenence=0, monthly_taxes=0):
    gross_rev = airbnb_daily_price * occupancy_rate * 365/12
    net_rev = gross_rev - monthly_maintenence - monthly_taxes
    profit = net_rev - monthly_mortgage
    roi = profit/monthly_mortgage
    roi = roi
    return roi

def set_location_desc(df,lat,long,geolocator):
    start_time = time.time()
    lat = str(lat)
    print("---Lat casted at value at %s seconds ---" % (time.time() - start_time))

    long = str(long)
    print("---Long casted at %s seconds ---" % (time.time() - start_time))

    location = geolocator.reverse(lat+","+long)
    print("---Get location at %s seconds ---" % (time.time() - start_time))

    address = location.raw['address']
    print("---Get address at %s seconds ---" % (time.time() - start_time))

    city = address.get('city', '')
    print("---Get city value at %s seconds ---" % (time.time() - start_time))

    state = address.get('state', '')
    print("---Get state value at %s seconds ---" % (time.time() - start_time))

    country = address.get('country')
    print("---Get country value at %s seconds ---" % (time.time() - start_time))
    country_code = address.get('country_code')
    print("---Get country_code value at %s seconds ---" % (time.time() - start_time))
    return city

def set_city(lat, long, geolocator):
    start_time = time.time()
    lat = str(lat)
    print("---Latitude casted to string at %s seconds ---" % (time.time() - start_time))

    long = str(long)
    print("---Longitude casted at %s seconds ---" % (time.time() - start_time))

    location = geolocator.reverse(lat+","+long)
    print("---Get location json from geopy at %s seconds ---" % (time.time() - start_time))

    address = location.raw['address']
    print("---Get address json at %s seconds ---" % (time.time() - start_time))

    city = address.get('city', '')
    print("---Get city value at %s seconds ---" % (time.time() - start_time))
    return city

def set_state(lat, long):
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="geoapiExercises")
    lat = str(lat)
    long = str(long)
    location = geolocator.reverse(Point(lat,long))
    address = location.raw['address']
    state = address.get('state', '')
    return state

def set_country(lat, long):
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="geoapiExercises")
    lat = str(lat)
    long = str(long)
    location = geolocator.reverse(Point(lat,long))
    address = location.raw['address']
    country = address.get('country')
    return country

def set_country_code(lat, long):
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="geoapiExercises")
    lat = str(lat)
    long = str(long)
    location = geolocator.reverse(lat+","+long)
    address = location.raw['address']
    country_code = address.get('country_code')
    return country_code

def set_zipcode(lat, long):
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="geoapiExercises")
    lat = str(lat)
    long = str(long)
    location = geolocator.reverse(Point(lat,long))
    address = location.raw['address']
    zipcode = address.get('postcode', '')
    return zipcode


In [3]:
"""
import os, json
import pandas as pd

# this finds our json files
path_to_json = '/Users/matthewgilgo/Downloads/airbnb_data/all_ids/all_ids.json/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

# here I define my pandas Dataframe with the columns I want to get from the json
jsons_data = pd.DataFrame(columns=['index', 'id', 'name', 'price', 'displayPrice','monthlyPriceFactor','weeklyPriceFactor','avgRating','reviewsCount','isNewListing','isSuperhost','lat','lng','personCapacity','size','beds','baths','title','property_type','town','top_lat','right_long','botton_lat','left_long','url'])

# we need both the json and an index number so use enumerate()
for ind, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        json_text = json.load(json_file)

        # here you need to know the layout of your json and each json has to have
        # the same structure (obviously not the structure I have here)
        index = json_text['features'][0]['index']
        id = json_text['features'][0]['id']
        name = json_text['features'][0]['name']
        price = json_text['features'][0]['price']
        displayPrice = json_text['features'][0]['displayPrice']
        monthlyPriceFactor = json_text['features'][0]['monthlyPriceFactor']
        weeklyPriceFactor = json_text['features'][0]['weeklyPriceFactor']
        avgRating = json_text['features'][0]['avgRating']
        reviewsCount = json_text['features'][0]['reviewsCount']
        isNewListing = json_text['features'][0]['isNewListing']
        isSuperhost = json_text['features'][0]['isSuperhost']
        lat = json_text['features'][0]['lat']
        lng = json_text['features'][0]['lng']
        personCapacity = json_text['features'][0]['personCapacity']
        size = json_text['features'][0]['size']
        beds = json_text['features'][0]['beds']
        baths = json_text['features'][0]['baths']
        title = json_text['features'][0]['title']
        property_type = json_text['features'][0]['property_type']
        town = json_text['features'][0]['town']
        top_lat = json_text['features'][0]['top_lat']
        right_long = json_text['features'][0]['right_long']
        botton_lat = json_text['features'][0]['botton_lat']
        left_long = json_text['features'][0]['left_long']
        url = json_text['features'][0]['url']          
        # here I push a list of data into a pandas DataFrame at row given by 'index'
        jsons_data.loc[ind] = [index,id,name,price,displayPrice,monthlyPriceFactor,weeklyPriceFactor,avgRating,reviewsCount,isNewListing,isSuperhost,lat,lng,personCapacity,size,beds,baths,title,property_type,town,top_lat,right_long,botton_lat,left_long,url]

# now that we have the pertinent json data in our DataFrame let's look at it
print(jsons_data)
"""

"\nimport os, json\nimport pandas as pd\n\n# this finds our json files\npath_to_json = '/Users/matthewgilgo/Downloads/airbnb_data/all_ids/all_ids.json/'\njson_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]\n\n# here I define my pandas Dataframe with the columns I want to get from the json\njsons_data = pd.DataFrame(columns=['index', 'id', 'name', 'price', 'displayPrice','monthlyPriceFactor','weeklyPriceFactor','avgRating','reviewsCount','isNewListing','isSuperhost','lat','lng','personCapacity','size','beds','baths','title','property_type','town','top_lat','right_long','botton_lat','left_long','url'])\n\n# we need both the json and an index number so use enumerate()\nfor ind, js in enumerate(json_files):\n    with open(os.path.join(path_to_json, js)) as json_file:\n        json_text = json.load(json_file)\n\n        # here you need to know the layout of your json and each json has to have\n        # the same structure (obviously not the structu

In [4]:
# Load in listing data
nc_dir = 'C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/listings/north_carolina'
first_file = ''
for file in os.listdir(nc_dir):
    first_file = os.path.join(nc_dir,file)
    break
listing_data = pd.read_parquet(first_file)
print(listing_data.shape)
for file in os.listdir(nc_dir):
    next_file = os.path.join(nc_dir,file)
    if next_file != first_file:
        next_listing = pd.read_parquet(next_file)
        listing_data = listing_data.append(next_listing)
print(listing_data.shape)

ne_dir = 'C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/listings/vt_nh'
for file in os.listdir(ne_dir):
    next_file = os.path.join(ne_dir,file)
    next_listing = pd.read_parquet(next_file)
    listing_data = listing_data.append(next_listing)
print(listing_data.shape)

miami_dir = 'C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/listings/miami'
for file in os.listdir(miami_dir):
    next_file = os.path.join(miami_dir,file)
    next_listing = pd.read_parquet(next_file)
    listing_data = listing_data.append(next_listing)
print(listing_data.shape)

# Load in occupancy data
occ_data = pd.read_parquet('C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/occupancy')

# Load in price data
price_data = pd.read_parquet('C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/prices')

(1, 25)


KeyboardInterrupt: 

In [ ]:
#listing_data = pd.read_parquet('C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/listings')
occ_data = pd.read_parquet('C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/occupancy')
price_data = pd.read_parquet('C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/prices')

In [ ]:
# Clean up listing_data
conditions = [
    listing_data['baths'].str.contains('shared', na=False),
    listing_data['baths'].str.contains('Shared', na=False)
]

values = ['Shared', 'Shared']

listing_data['baths_type'] = np.select(conditions, values, default='Private')

conditions_loc = [
    listing_data['top_lat'] > 35.5,
    listing_data['top_lat'] < 28
]

values_loc = ['New England', 'Miami']

listing_data['Region'] = np.select(conditions_loc, values_loc, default='Carolinas')


conditions_halfbath = [
    listing_data['baths'].str.contains('Half-bath', na=False),
    listing_data['baths'].str.contains('Shared half-bath', na=False),
    listing_data['baths'].str.contains('Private half-bath', na=False)
]

values_halfbath = [0.5,0.5,0.5]

listing_data['baths'] = np.select(conditions_halfbath, values_halfbath, default=listing_data['baths'])

listing_data['baths_no'] = listing_data['baths'].str.split(' ').str[0]
listing_data['beds_no'] = listing_data['beds'].str.split(' ').str[0]
listing_data['guest_no'] = listing_data['title'].str.split(' ').str[0]
listing_data['guest_no'] = listing_data['guest_no'].astype('float')
listing_data['id'] = listing_data['id'].astype('float64')
listing_data['id'] = listing_data['id'].astype('str')
listing_data['town'] = listing_data['town'].str.lower()
listing_data = listing_data[listing_data.lat.isna() == False]
#listing_data['city_geopy'] = listing_data.apply(lambda row: set_city(row['lat'], row['lng']), axis=1)
#listing_data['state_geopy'] = listing_data.apply(lambda row: set_state(row['lat'], row['lng']), axis=1)
#listing_data['country_geopy'] = listing_data.apply(lambda row: set_country(row['lat'], row['lng']), axis=1)
#listing_data['country_code_geopy'] = listing_data.apply(lambda row: set_country_code(row['lat'], row['lng']), axis=1)
#listing_data['zipcode_geopy'] = listing_data.apply(lambda row: set_zipcode(row['lat'], row['lng']), axis=1)

# Clean up occ_data
#occ_data['id'] = occ_data['id'].astype('int')
occ_data['id'] = occ_data['id'].astype('str')

# Clean up price_data
#price_data['id'] = price_data['id'].astype('int')
price_data['id'] = price_data['id'].astype('str')

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
#listing_data['city_geopy'] = listing_data.apply(lambda row: set_city(row['lat'], row['lng'], geolocator), axis=1)
print(set_city(35, -76, geolocator))


In [ ]:
price_data_grouped = price_data.groupby('id').mean()

In [ ]:
list_n_price_data = listing_data.merge(price_data_grouped, how = 'left', on='id')

In [ ]:
# Future way to pull down listing data
"""
path = 'C:/Users/mattg/Desktop/Hobbies/bucket_data'
df = pd.DataFrame(columns=['observation'])
for directory in os.listdir(path):
    if os.path.isdir(directory):
        for filename in os.listdir(directory):
            with open(os.path.join(directory, filename)) as f:
                observation = f.read()
                print('hi')
                current_df = pd.DataFrame({'observation': [observation]})
                df = df.append(current_df, ignore_index=True)
"""

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(listing_data, alpha=0.2, figsize=(15,15))

In [ ]:
#null_town = all_ne_data[all_ne_data['town'].isna()]
#null_town.to_csv('nulltowns.csv')

In [ ]:
#bed_bath_prices = all_ne_data.groupby(['beds', 'baths'])['price'].mean().reset_index()
#bed_bath_counts = all_ne_data.groupby(['beds', 'baths'])['price'].count().reset_index()
bed_bath_prices = pd.read_csv('data/bed_bath_avgs_wbeds.csv')
bed_bath_counts = pd.read_csv('data/bed_bath_counts_wbeds.csv')


In [ ]:

bed_bath_prices_rev2 = listing_data.groupby(['beds_no', 'baths_no', 'baths_type'])['price'].mean().reset_index()
bed_bath_counts_rev2 = listing_data.groupby(['beds_no', 'baths_no', 'baths_type'])['price'].count().reset_index()
bed_bath_prices_rev2 = bed_bath_prices_rev2.dropna(axis=0, subset=['beds_no'])
bed_bath_counts_rev2 = bed_bath_counts_rev2.dropna(axis=0, subset=['beds_no'])
bed_bath_prices_rev2.to_csv('grouped_data/bed_bath_prices_rev2.csv')
bed_bath_prices_rev2.to_csv('grouped_data/bed_bath_counts_rev2.csv')


In [ ]:
bed_bath_prices_rev2 = pd.read_csv('grouped_data/bed_bath_prices_rev2.csv')
bed_bath_counts_rev2 = pd.read_csv('grouped_data/bed_bath_counts_rev2.csv')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.scatterplot(x=bed_bath_prices_rev2['beds_no'], y=bed_bath_prices_rev2['baths_no'], style=bed_bath_prices_rev2['baths_type'], hue=bed_bath_prices_rev2['price'])
plt.title('Price vs Bed/Bath Combinations')
plt.xlabel('Baths')
plt.ylabel('Beds')

In [ ]:
avgs = listing_data.groupby(['guest_no','Region'])['price'].mean().reset_index()
import seaborn as sns
sns.scatterplot(x=avgs['guest_no'], y=avgs['price'], hue = avgs['Region'])
plt.title('Price vs Guests')
plt.xlabel('Guests')
plt.ylabel('Avg Price')


In [ ]:
avgRats = listing_data.groupby(['guest_no','Region'])['avgRating'].mean().reset_index()
sns.scatterplot(x=avgRats['guest_no'], y=avgRats['avgRating'], hue = avgRats['Region'])
plt.title('Avg Rating vs Guests')
plt.xlabel('Guests')
plt.ylabel('Avg Rating')


In [ ]:
sns.boxplot(x="guest_no", y="price", data=listing_data)
plt.ylim([0,2000])

In [ ]:
counts = listing_data.groupby(['guest_no'])['id'].count().reset_index()
sns.scatterplot(x=counts['guest_no'], y=counts['id'])
plt.title('Count of IDs vs Guests')
plt.xlabel('# of Guests')
plt.ylabel('ID Count')

In [ ]:
counts_loc = listing_data.groupby(['guest_no','Region'])['id'].count().reset_index()
sns.scatterplot(x=counts_loc['guest_no'], y=counts_loc['id'], hue = counts_loc['Region'])
plt.title('Count of IDs vs Guests')
plt.xlabel('# of Guests')
plt.ylabel('ID Count')

In [ ]:
sns.boxplot(x="guest_no", y="avgRating", data=listing_data)
#plt.ylim([0,2000])

In [ ]:
listing_data.columns

In [ ]:
import folium
#listing_locations = all_ne_data[["lat", "lng", "name"]]
#listing_locations = listing_locations.dropna()
listing_data = listing_data.dropna()
m = folium.Map(location=[listing_data.lat.mean(), listing_data.lng.mean()], zoom_start=3)

tooltip = "Click me!"
counter = 0
for index, location_info in listing_data.iterrows():
#for index, location_info in listing_locations.iterrows():
    if location_info.reviewsCount > 100:
        folium.Marker(
            [location_info["lat"], location_info["lng"]], popup=location_info["price"], icon=folium.Icon(color="darkgreen")
        ).add_to(m)
    else:
        folium.Marker(
            [location_info["lat"], location_info["lng"]], popup=location_info["price"], icon=folium.Icon(color="blue")
        ).add_to(m)        
    counter += 1
    if counter == 1000:
        break

#m.save('map.html')

In [ ]:
m

In [ ]:
listing_data.isSuperhost.unique()

In [ ]:
listing_data.shape

In [ ]:
listing_data

In [ ]:
nc = listing_data[listing_data['top_lat'] < 35.5]

In [ ]:
nc.shape

In [ ]:
nc.town.unique()

In [ ]:
from glob import glob
import numpy as np
import folium
from folium import plugins
from folium.plugins import HeatMap

lon, lat = -77.5, 34.2
zoom_start = 5

#data = (
#    np.random.normal(size=(100, 3)) *
#    np.array([[1, 1, 1]]) +
#    np.array([[48, 5, 1]])
#).tolist()
m = folium.Map([48, 5], tiles='stamentoner', zoom_start=6)
#nc = nc.dropna()
HeatMap(listing_data[['lat','lng']].groupby(['lat','lng']).sum().reset_index().values.tolist(), radius=7, max_zoom=10).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))
folium.LayerControl().add_to(m)

In [ ]:
m

In [42]:
import plotly.express as px
listing_fig = px.density_mapbox(listing_data, lat='lat', lon='lng', z='id', radius=10,
                        center=dict(lat=listing_data.lat.mean(), lon=listing_data.lng.mean()), zoom=3,
                        mapbox_style="stamen-terrain")
#fig.show()
listing_fig.update_layout(title_text="Heatmap for Airbnb Listings")

listing_fig.write_html("reports/listing_heatmap.html")

In [43]:
price_fig = px.density_mapbox(listing_data, lat='lat', lon='lng', z='price', radius=10,
                        center=dict(lat=listing_data.lat.mean(), lon=listing_data.lng.mean()), zoom=3,
                        mapbox_style="stamen-terrain")
#fig.show()
price_fig.update_layout(title_text="Heatmap for Airbnb Price")
price_fig.write_html("reports/price_heatmap.html")

In [ ]:
pip install us

In [44]:
city_data = pd.read_csv('resource_data/uscities.csv')

In [ ]:
city_data.columns

In [ ]:
test_zip = 91311
city_name = 'Los Angeles'
    zip_codes = city_data[city_data['city'] == city_name].zips

In [45]:
home_value = pd.read_csv('resource_data/home_value_zillow.csv')

In [10]:
city_home = city_data.merge(home_value, left_on = ['city','state_id'], right_on = ['RegionName','State'])

In [ ]:
city_home

In [ ]:
city_home.shape

In [11]:
city_home['city'] = city_home['city'].str.lower()
lists_per_city = listing_data.groupby(['town'])['id'].count().reset_index()
lists_per_city.rename(columns = {'id':'num_lists_in_town'}, inplace = True)
median_price_per_city = listing_data.groupby(['town'])['price'].median().reset_index()
median_price_per_city.rename(columns = {'price':'median_price_per_city'}, inplace = True)

# Calculate occupancy rate
occ_rate = occ_data.groupby('id')['available'].apply(lambda row: np.sum(row)/len(row))
listing_occ_rate = listing_data.merge(occ_rate, on = 'id')


# Calculate pricing avgs
cleaning_fee = price_data.groupby(['id'])['cleaning_fee'].median().reset_index()
cleaning_fee.rename(columns = {'cleaning_fee':'median_cleaning_fee'}, inplace = True)
service_fee = price_data.groupby(['id'])['service_fee'].median().reset_index()
service_fee.rename(columns = {'service_fee':'median_service_fee'}, inplace = True)
#total_price = price_data.groupby(['id'])['total_price'].median().reset_index()
#total_price.rename(columns = {'total_price':'median_total_price'}, inplace = True)

listing_occ_rate_fees = listing_occ_rate.merge(cleaning_fee, on='id')
listing_occ_rate_fees = listing_occ_rate_fees.merge(service_fee, on='id')
#listing_occ_rate_fees = listing_occ_rate_fees.merge(total_price, on='id')

median_occ_rate_per_city = listing_occ_rate_fees.groupby(['town'])['available'].median().reset_index()
median_occ_rate_per_city.rename(columns = {'available':'occupancy_rate'}, inplace = True)

median_cleaning_fee_per_city = listing_occ_rate_fees.groupby(['town'])['median_cleaning_fee'].median().reset_index()
median_service_fee_per_city = listing_occ_rate_fees.groupby(['town'])['median_service_fee'].median().reset_index()
#median_total_price_per_city = listing_occ_rate_fees.groupby(['town'])['median_total_price'].median().reset_index()
#fees_and_price = cleaning_fee.merge(service_fee, on = 'id')
#fees_and_price = fees_and_price.merge(total_price, on = 'id')


In [12]:
lists_town_price = lists_per_city.merge(median_price_per_city, left_on = 'town', right_on = 'town')
lists_town_price = lists_town_price.merge(median_cleaning_fee_per_city, left_on = 'town', right_on = 'town')
lists_town_price = lists_town_price.merge(median_service_fee_per_city, left_on = 'town', right_on = 'town')
#lists_town_price = lists_town_price.merge(median_total_price_per_city, left_on = 'town', right_on = 'town')
lists_town_price['median_total_price'] = lists_town_price['median_price_per_city'] + lists_town_price['median_cleaning_fee'] + lists_town_price['median_service_fee']
lists_town_price_occ = lists_town_price.merge(median_occ_rate_per_city, left_on = 'town', right_on = 'town')
lists_town_home_price_occ = lists_town_price_occ.merge(city_home, left_on = 'town', right_on = 'city')

In [ ]:
lists_town_home_price_occ.head(5)

In [67]:
lists_town_home_price_occ_vt = lists_town_home_price_occ[(lists_town_home_price_occ['state_id'] == 'VT')]
lists_town_home_price_occ_nh = lists_town_home_price_occ[(lists_town_home_price_occ['state_id'] == 'NH')]
lists_town_home_price_occ_nc = lists_town_home_price_occ[(lists_town_home_price_occ['state_id'] == 'NC')]
lists_town_home_price_occ_sc = lists_town_home_price_occ[(lists_town_home_price_occ['state_id'] == 'SC')]
lists_town_home_price_occ_fl = lists_town_home_price_occ[(lists_town_home_price_occ['state_id'] == 'FL')]
lists_town_home_price_occ_fl = lists_town_home_price_occ[(lists_town_home_price_occ['state_id'] == 'ME')]


lists_town_home_price_occ_all = lists_town_home_price_occ_vt.append(lists_town_home_price_occ_nh)
lists_town_home_price_occ_all = lists_town_home_price_occ_all.append(lists_town_home_price_occ_nc)
lists_town_home_price_occ_all = lists_town_home_price_occ_all.append(lists_town_home_price_occ_sc)
lists_town_home_price_occ_all = lists_town_home_price_occ_all.append(lists_town_home_price_occ_fl)
lists_town_home_price_occ_all = lists_town_home_price_occ_all.append(lists_town_home_price_occ_me)



In [ ]:
lists_town_home_price_occ_all

In [68]:
import plotly.graph_objects as go

price_v_numlist_fig = go.Figure(data=[
    go.Bar(name='Price', x=lists_town_home_price_occ_all['town'], y=lists_town_home_price_occ_all['median_price_per_city']),
    go.Bar(name='# of Listings', x=lists_town_home_price_occ_all['town'], y=lists_town_home_price_occ_all['num_lists_in_town'])
])
# Change the bar mode
price_v_numlist_fig.update_layout(title_text='Price vs Number of Airbnb Listings', barmode='group')
price_v_numlist_fig.show()

In [ ]:
print(calculate_mortgage(100000, 3, 15))

In [69]:
lists_town_home_price_occ_all['avg_30_yr_mort'] = lists_town_home_price_occ_all.apply(lambda row: calculate_mortgage(row['2022-04-30'], 5, 30), axis=1)

In [70]:
lists_town_home_price_occ_all['median_ROI'] = lists_town_home_price_occ_all.apply(lambda row: calculate_roi(row['median_price_per_city'], row['occupancy_rate'], row['avg_30_yr_mort']), axis=1)

In [ ]:
lists_town_home_price_occ_all.head(5)

In [71]:
import plotly.graph_objects as go

roi_fig = go.Figure(data=[
    go.Bar(name='ROI', x=lists_town_home_price_occ_all['town'], y=lists_town_home_price_occ_all['median_ROI']),
])
# Change the bar mode
roi_fig.update_layout(title_text = 'ROI by City', barmode='group')
roi_fig.show()

In [ ]:
lists_town_home_price_occ_all['median_cleaning_fee_per_city'].sort_values(ascending=False)

In [72]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

roi_subplots = make_subplots(rows=3, cols=2)

lists_town_home_price_occ_vt = lists_town_home_price_occ_all[(lists_town_home_price_occ_all['state_id'] == 'VT')]
lists_town_home_price_occ_nh = lists_town_home_price_occ_all[(lists_town_home_price_occ_all['state_id'] == 'NH')]
lists_town_home_price_occ_nc = lists_town_home_price_occ_all[(lists_town_home_price_occ_all['state_id'] == 'NC')]
lists_town_home_price_occ_sc = lists_town_home_price_occ_all[(lists_town_home_price_occ_all['state_id'] == 'SC')]
lists_town_home_price_occ_fl = lists_town_home_price_occ_all[(lists_town_home_price_occ_all['state_id'] == 'FL')]
lists_town_home_price_occ_me = lists_town_home_price_occ_all[(lists_town_home_price_occ_all['state_id'] == 'ME')]

roi_subplots.append_trace(
    go.Bar(x=lists_town_home_price_occ_vt['town'],
    y=lists_town_home_price_occ_vt['median_ROI'], name="VT"
), row=1, col=1)

roi_subplots.append_trace(go.Bar(
    x=lists_town_home_price_occ_nh['town'],
    y=lists_town_home_price_occ_nh['median_ROI'], name="NH"
), row=1, col=2)

roi_subplots.append_trace(go.Bar(
    x=lists_town_home_price_occ_nc['town'],
    y=lists_town_home_price_occ_nc['median_ROI'], name="NC"
), row=2, col=1)

roi_subplots.append_trace(go.Bar(
    x=lists_town_home_price_occ_sc['town'],
    y=lists_town_home_price_occ_sc['median_ROI'], name="SC"
), row=2, col=2)

roi_subplots.append_trace(go.Bar(
    x=lists_town_home_price_occ_fl['town'],
    y=lists_town_home_price_occ_fl['median_ROI'], name="FL"
), row=3, col=1)

roi_subplots.append_trace(go.Bar(
    x=lists_town_home_price_occ_me['town'],
    y=lists_town_home_price_occ_me['median_ROI'], name="ME"
), row=3, col=2)


roi_subplots.update_layout(height = 1100, title_text="ROI for Cities in each State", barmode='group')
roi_subplots.show()

In [ ]:
vt_pricing_fig = go.Figure(data=[
    go.Bar(name='Display Price', x=lists_town_home_price_occ_vt['town'], y=lists_town_home_price_occ_vt['median_price_per_city']),
    go.Bar(name='Cleaning Fee', x=lists_town_home_price_occ_vt['town'], y=lists_town_home_price_occ_vt['median_cleaning_fee']),
    go.Bar(name='Service Fee', x=lists_town_home_price_occ_vt['town'], y=lists_town_home_price_occ_vt['median_service_fee']),
    #go.Bar(name='Total Price', x=lists_town_home_price_occ_vt['town'], y=lists_town_home_price_occ_vt['median_total_price'])
])
# Change the bar mode
vt_pricing_fig.update_layout(title_text='Pricing Data by City (Vermont)', barmode='stack')
vt_pricing_fig.show()


In [ ]:
nh_pricing_fig = go.Figure(data=[
    go.Bar(name='Display Price', x=lists_town_home_price_occ_nh['town'], y=lists_town_home_price_occ_nh['median_price_per_city']),
    go.Bar(name='Cleaning Fee', x=lists_town_home_price_occ_nh['town'], y=lists_town_home_price_occ_nh['median_cleaning_fee']),
    go.Bar(name='Service Fee', x=lists_town_home_price_occ_nh['town'], y=lists_town_home_price_occ_nh['median_service_fee']),
    #go.Bar(name='Total Price', x=lists_town_home_price_occ_nh['town'], y=lists_town_home_price_occ_nh['median_total_price'])
])
# Change the bar mode
nh_pricing_fig.update_layout(title_text='Pricing Data by City (New Hampshire)', barmode='stack')
nh_pricing_fig.show()

In [ ]:
nc_pricing_fig = go.Figure(data=[
    go.Bar(name='Display Price', x=lists_town_home_price_occ_nc['town'], y=lists_town_home_price_occ_nc['median_price_per_city']),
    go.Bar(name='Cleaning Fee', x=lists_town_home_price_occ_nc['town'], y=lists_town_home_price_occ_nc['median_cleaning_fee']),
    go.Bar(name='Service Fee', x=lists_town_home_price_occ_nc['town'], y=lists_town_home_price_occ_nc['median_service_fee']),
    #go.Bar(name='Total Price', x=lists_town_home_price_occ_nc['town'], y=lists_town_home_price_occ_nc['median_total_price'])
])
# Change the bar mode
nc_pricing_fig.update_layout(title_text='Pricing Data by City (North Carolina)', barmode='stack')
nc_pricing_fig.show()

In [ ]:
sc_pricing_fig = go.Figure(data=[
    go.Bar(name='Display Price', x=lists_town_home_price_occ_sc['town'], y=lists_town_home_price_occ_sc['median_price_per_city']),
    go.Bar(name='Cleaning Fee', x=lists_town_home_price_occ_sc['town'], y=lists_town_home_price_occ_sc['median_cleaning_fee']),
    go.Bar(name='Service Fee', x=lists_town_home_price_occ_sc['town'], y=lists_town_home_price_occ_sc['median_service_fee']),
    #go.Bar(name='Total Price', x=lists_town_home_price_occ_sc['town'], y=lists_town_home_price_occ_sc['median_total_price'])
])
# Change the bar mode
sc_pricing_fig.update_layout(title_text='Pricing Data by City (South Carolina)', barmode='stack')
sc_pricing_fig.show()

In [18]:
# grouping by town and guests

city_home['city'] = city_home['city'].str.lower()
lists_per_city_guest = listing_data.groupby(['town', 'guest_no'])['id'].count().reset_index()
lists_per_city_guest.rename(columns = {'id':'num_lists_in_town'}, inplace = True)
median_price_per_city_guest = listing_data.groupby(['town', 'guest_no'])['price'].median().reset_index()
median_price_per_city_guest.rename(columns = {'price':'median_price_per_city'}, inplace = True)

# Calculate occupancy rate
occ_rate = occ_data.groupby('id')['available'].apply(lambda row: np.sum(row)/len(row))
listing_occ_rate = listing_data.merge(occ_rate, on = 'id')


# Calculate pricing avgs
cleaning_fee = price_data.groupby(['id'])['cleaning_fee'].median().reset_index()
cleaning_fee.rename(columns = {'cleaning_fee':'median_cleaning_fee'}, inplace = True)
service_fee = price_data.groupby(['id'])['service_fee'].median().reset_index()
service_fee.rename(columns = {'service_fee':'median_service_fee'}, inplace = True)
#total_price = price_data.groupby(['id'])['total_price'].median().reset_index()
#total_price.rename(columns = {'total_price':'median_total_price'}, inplace = True)

listing_occ_rate_fees = listing_occ_rate.merge(cleaning_fee, on='id')
listing_occ_rate_fees = listing_occ_rate_fees.merge(service_fee, on='id')
#listing_occ_rate_fees = listing_occ_rate_fees.merge(total_price, on='id')

median_occ_rate_per_city_guest = listing_occ_rate_fees.groupby(['town', 'guest_no'])['available'].median().reset_index()
median_occ_rate_per_city_guest.rename(columns = {'available':'occupancy_rate'}, inplace = True)

median_cleaning_fee_per_city_guest = listing_occ_rate_fees.groupby(['town', 'guest_no'])['median_cleaning_fee'].median().reset_index()
median_service_fee_per_city_guest = listing_occ_rate_fees.groupby(['town', 'guest_no'])['median_service_fee'].median().reset_index()


lists_town_price_guest = lists_per_city_guest.merge(median_price_per_city_guest, left_on = 'town', right_on = 'town')
lists_town_price_guest = lists_town_price_guest.merge(median_cleaning_fee_per_city_guest, left_on = 'town', right_on = 'town')
lists_town_price_guest = lists_town_price_guest.merge(median_service_fee_per_city_guest, left_on = 'town', right_on = 'town')
#lists_town_price = lists_town_price.merge(median_total_price_per_city, left_on = 'town', right_on = 'town')
lists_town_price_guest['median_total_price'] = lists_town_price_guest['median_price_per_city'] + lists_town_price_guest['median_cleaning_fee'] + lists_town_price_guest['median_service_fee']
lists_town_price_occ_guest = lists_town_price_guest.merge(median_occ_rate_per_city_guest, left_on = 'town', right_on = 'town')
city_home_needed_cols = city_home[['city','state_id','population','2022-04-30']]
lists_town_home_price_occ_guest = lists_town_price_occ_guest.merge(city_home_needed_cols, how='inner', left_on = 'town', right_on = 'city')

In [13]:
lists_town_home_price_occ_guest.head(5)

,town,guest_no_x,num_lists_in_town,guest_no_y,median_price_per_city,guest_no_x,median_cleaning_fee,guest_no_y,median_service_fee,median_total_price,guest_no,occupancy_rate,city,state_id,population,2022-04-30
0,south londonderry,11.0,1,11.0,2500.0,11.0,150.0,11.0,374.0,3024.0,11.0,0.978082,NaN,NaN,NaN,NaN
1,acton,2.0,4,2.0,155.0,2.0,50.0,2.0,29.0,234.0,2.0,0.317260,acton,CA,7054.0,885107.0
2,acton,2.0,4,2.0,155.0,2.0,50.0,2.0,29.0,234.0,6.0,0.071233,acton,CA,7054.0,885107.0
3,acton,2.0,4,2.0,155.0,2.0,50.0,2.0,29.0,234.0,10.0,0.312329,acton,CA,7054.0,885107.0
4,acton,2.0,4,2.0,155.0,2.0,50.0,6.0,74.0,279.0,2.0,0.317260,acton,CA,7054.0,885107.0


In [46]:
# grouping by town and guests

city_home['city'] = city_home['city'].str.lower()
print("city home shape: " + str(city_home.shape))
lists_per_city_guest = listing_data.groupby(['town', 'guest_no'])['id'].count().reset_index()
lists_per_city_guest.rename(columns = {'id':'num_lists_in_town'}, inplace = True)
print("lists_per_city_guest shape: " + str(lists_per_city_guest.shape))
median_price_per_city_guest = listing_data.groupby(['town', 'guest_no'])['price'].median().reset_index()
median_price_per_city_guest.rename(columns = {'price':'median_price_per_city'}, inplace = True)
print("median_price_per_city_guest shape: " + str(median_price_per_city_guest.shape))


# Calculate occupancy rate
occ_rate = occ_data.groupby('id')['available'].apply(lambda row: np.sum(row)/len(row))
listing_occ_rate = listing_data.merge(occ_rate, on = 'id')
print("listing_occ_rate shape: " + str(listing_occ_rate.shape))



# Calculate pricing avgs
cleaning_fee = price_data.groupby(['id'])['cleaning_fee'].median().reset_index()
cleaning_fee.rename(columns = {'cleaning_fee':'median_cleaning_fee'}, inplace = True)
service_fee = price_data.groupby(['id'])['service_fee'].median().reset_index()
service_fee.rename(columns = {'service_fee':'median_service_fee'}, inplace = True)
#total_price = price_data.groupby(['id'])['total_price'].median().reset_index()
#total_price.rename(columns = {'total_price':'median_total_price'}, inplace = True)

listing_occ_rate_fees = listing_occ_rate.merge(cleaning_fee, on='id')
print("listing_occ_rate_fees shape: " + str(listing_occ_rate_fees.shape))
listing_occ_rate_fees = listing_occ_rate_fees.merge(service_fee, on='id')
print("listing_occ_rate_fees shape: " + str(listing_occ_rate_fees.shape))

#listing_occ_rate_fees = listing_occ_rate_fees.merge(total_price, on='id')

median_occ_rate_per_city_guest = listing_occ_rate_fees.groupby(['town', 'guest_no'])['available'].median().reset_index()
median_occ_rate_per_city_guest.rename(columns = {'available':'occupancy_rate'}, inplace = True)
print("median_occ_rate_per_city_guest shape: " + str(median_occ_rate_per_city_guest.shape))


median_cleaning_fee_per_city_guest = listing_occ_rate_fees.groupby(['town', 'guest_no'])['median_cleaning_fee'].median().reset_index()
print("median_occ_rate_per_city_guest shape: " + str(median_occ_rate_per_city_guest.shape))

median_service_fee_per_city_guest = listing_occ_rate_fees.groupby(['town', 'guest_no'])['median_service_fee'].median().reset_index()
print("median_service_fee_per_city_guest shape: " + str(median_service_fee_per_city_guest.shape))


lists_town_price_guest = lists_per_city_guest.merge(median_price_per_city_guest, on = ['town', 'guest_no'])
print("lists_town_price_guest shape: " + str(lists_town_price_guest.shape))

lists_town_price_guest = lists_town_price_guest.merge(median_cleaning_fee_per_city_guest, on = ['town', 'guest_no'])
print("lists_town_price_guest shape: " + str(lists_town_price_guest.shape))

lists_town_price_guest = lists_town_price_guest.merge(median_service_fee_per_city_guest, on = ['town', 'guest_no'])
print("lists_town_price_guest shape: " + str(lists_town_price_guest.shape))

#lists_town_price = lists_town_price.merge(median_total_price_per_city, left_on = 'town', right_on = 'town')
lists_town_price_guest['median_total_price'] = lists_town_price_guest['median_price_per_city'] + lists_town_price_guest['median_cleaning_fee'] + lists_town_price_guest['median_service_fee']
print("lists_town_price_guest shape: " + str(lists_town_price_guest.shape))

lists_town_price_occ_guest = lists_town_price_guest.merge(median_occ_rate_per_city_guest, on = ['town', 'guest_no'])
print("lists_town_price_guest shape: " + str(lists_town_price_guest.shape))

city_home_needed_cols = city_home[['city','state_id','population','2022-04-30']]
print("city_home_needed_cols shape: " + str(city_home_needed_cols.shape))

lists_town_home_price_occ_guest = lists_town_price_occ_guest.merge(city_home_needed_cols, how='inner', left_on = 'town', right_on = 'city')
print("lists_town_home_price_occ_guest shape: " + str(lists_town_home_price_occ_guest.shape))

lists_town_home_price_occ_guest['avg_30_yr_mort'] = lists_town_home_price_occ_guest.apply(lambda row: calculate_mortgage(row['2022-04-30'], 5, 30), axis=1)

lists_town_home_price_occ_guest['median_ROI'] = lists_town_home_price_occ_guest.apply(lambda row: calculate_roi(row['median_price_per_city'], row['occupancy_rate'], row['avg_30_yr_mort']), axis=1)


wb_data = lists_town_home_price_occ_guest[lists_town_home_price_occ_guest['town'] == 'wrightsville beach']
print("wb_data shape: " + str(wb_data.shape))



city home shape: (18381, 293)
lists_per_city_guest shape: (3456, 3)
median_price_per_city_guest shape: (3456, 3)
listing_occ_rate shape: (3978, 31)
listing_occ_rate_fees shape: (3978, 32)
listing_occ_rate_fees shape: (3978, 33)
median_occ_rate_per_city_guest shape: (1162, 3)
median_occ_rate_per_city_guest shape: (1162, 3)
median_service_fee_per_city_guest shape: (1162, 3)
lists_town_price_guest shape: (3456, 4)
lists_town_price_guest shape: (1162, 5)
lists_town_price_guest shape: (1162, 6)
lists_town_price_guest shape: (1162, 7)
lists_town_price_guest shape: (1162, 7)
city_home_needed_cols shape: (18381, 4)
lists_town_home_price_occ_guest shape: (3310, 12)
wb_data shape: (9, 14)


In [45]:
wb_fig = go.Figure(data=[
    go.Bar(name='Total Price', x=wb_data['guest_no'], y=wb_data['median_total_price'], yaxis='y', offsetgroup=1),
    go.Bar(name='Occupancy Rate', x=wb_data['guest_no'], y=wb_data['occupancy_rate'], yaxis='y2', offsetgroup=2)
],
    layout={
        'xaxis': {'title': '# of Guests'},
        'yaxis': {'title': 'Total Price'},
        'yaxis2': {'title': 'Occupancy Rate', 'overlaying': 'y', 'side': 'right'}
    }
)

# Change the bar mode
wb_fig.update_layout(title_text='Wrightsville Beach Price and Occupancy by # of Guests', barmode='group')
wb_fig.show()

In [49]:
wb_fig = go.Figure(data=[
    go.Bar(name='Total Price', x=wb_data['guest_no'], y=wb_data['median_total_price'], yaxis='y', offsetgroup=1),
    go.Bar(name='Occupancy Rate', x=wb_data['guest_no'], y=wb_data['occupancy_rate'], yaxis='y2', offsetgroup=2),
],
    layout={
        'xaxis': {'title': '# of Guests'},
        'yaxis': {'title': 'Total Price'},
        'yaxis2': {'title': 'Occupancy Rate', 'overlaying': 'y', 'side': 'right'}
    }
)

# Change the bar mode
wb_fig.update_layout(title_text='Wrightsville Beach Price and Occupancy by # of Guests', barmode='group')
wb_fig.show()

In [51]:
append_fig_to_html([wb_fig])

In [95]:
#nh_data = lists_town_home_price_occ_guest[lists_town_home_price_occ_guest['town'] == 'wrightsville beach']
lists_town_home_price_occ_guest.columns

Index(['town', 'guest_no', 'num_lists_in_town', 'median_price_per_city',
       'median_cleaning_fee', 'median_service_fee', 'median_total_price',
       'occupancy_rate', 'city', 'state_id', 'population', '2022-04-30',
       'avg_30_yr_mort', 'median_ROI'],
      dtype='object')

In [62]:
c

,town,guest_no,num_lists_in_town,median_price_per_city,median_cleaning_fee,median_service_fee,median_total_price,occupancy_rate,city,state_id,population,2022-04-30,avg_30_yr_mort,median_ROI
0,acton,2.0,4,155.0,50.0,29.0,234.0,0.317260,acton,CA,7054,885107.0,4751.45,-0.685201
1,acton,6.0,5,280.0,175.0,74.0,529.0,0.071233,acton,CA,7054,885107.0,4751.45,-0.872320
2,acton,10.0,4,300.0,150.0,64.0,514.0,0.312329,acton,CA,7054,885107.0,4751.45,-0.400183
3,albany,2.0,2,119.0,NaN,20.0,NaN,0.534247,albany,NY,590823,259080.0,1390.80,0.390387
4,albany,2.0,2,119.0,NaN,20.0,NaN,0.534247,albany,GA,89323,109595.0,588.33,2.286846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3305,york,14.0,12,707.0,650.0,215.0,1572.0,0.789041,york,AL,2448,47528.0,255.14,65.504664
3306,york,16.0,7,1090.0,485.0,241.5,1816.5,0.568219,york,PA,236818,229906.0,1234.19,14.264127
3307,york,16.0,7,1090.0,485.0,241.5,1816.5,0.568219,york,SC,8287,318429.0,1709.40,10.020729
3308,york,16.0,7,1090.0,485.0,241.5,1816.5,0.568219,york,NE,7810,165233.0,887.01,20.238581


In [66]:
lists_town_home_price_occ_guest_nc = lists_town_home_price_occ_guest[(lists_town_home_price_occ_guest['state_id'] == 'NC')]


num_lists_pivot = lists_town_home_price_occ_guest_nc.pivot(index='town', columns='guest_no', values='num_lists_in_town')

In [1]:
import dash
import dash_html_components as html
import plotly.graph_objects as go
import dash_core_components as dcc
import plotly.express as px
from dash.dependencies import Input, Output


app = dash.Dash()

df = px.data.stocks()


app.layout = html.Div(id = 'parent', children = [
    html.H1(id = 'H1', children = 'Styling using html components', style = {'textAlign':'center',\
                                            'marginTop':40,'marginBottom':40}),

        dcc.Dropdown( id = 'dropdown',
        options = [
            {'label':'Google', 'value':'GOOG' },
            {'label': 'Apple', 'value':'AAPL'},
            {'label': 'Amazon', 'value':'AMZN'},
            ],
        value = 'GOOG'),
        dcc.Graph(id = 'bar_plot')
    ])
    
    
@app.callback(Output(component_id='bar_plot', component_property= 'figure'),
              [Input(component_id='dropdown', component_property= 'value')])
def graph_update(dropdown_value):
    print(dropdown_value)
    fig = go.Figure([go.Bar(x = df['date'], y = df['{}'.format(dropdown_value)],\
                     line = dict(color = 'firebrick', width = 4))
                     ])
    
    fig.update_layout(title = 'Stock prices over time',
                      xaxis_title = 'Dates',
                      yaxis_title = 'Prices'
                      )
    return fig  



if __name__ == '__main__': 
    app.run_server()

C:\Users\mattg\AppData\Local\Temp\ipykernel_15692\2909838520.py:2: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\mattg\AppData\Local\Temp\ipykernel_15692\2909838520.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
127.0.0.1 - - [22/May/2022 18:12:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2022 18:12:25] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2022 18:12:25] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2022 18:12:25] "GET /_favicon.ico?v=2.4.1 HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2022 18:12:25] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [22/May/2022 18:12:25] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [22/May/2022 18:12:25] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


GOOG
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\mattg\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\mattg\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\mattg\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\mattg\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "c:\Users\mattg\AppData\Local\Programs\Python\Python310\lib\site-packages\dash\dash.py", line 1372, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "c:\Users\mattg\AppDa

127.0.0.1 - - [22/May/2022 18:12:25] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [22/May/2022 18:12:26] "GET /_favicon.ico?v=2.4.1 HTTP/1.1" 200 -


AAPL
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\mattg\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\mattg\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\mattg\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\mattg\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "c:\Users\mattg\AppData\Local\Programs\Python\Python310\lib\site-packages\dash\dash.py", line 1372, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "c:\Users\mattg\AppDa

127.0.0.1 - - [22/May/2022 18:12:27] "POST /_dash-update-component HTTP/1.1" 500 -


In [ ]:
        dcc.Dropdown( id = 'dropdown',
        options = [
            {'label':'Google', 'value':'GOOG' },
            {'label': 'Apple', 'value':'AAPL'},
            {'label': 'Amazon', 'value':'AMZN'},
            ],
        value = 'GOOG'),
        dcc.Graph(id = 'bar_plot')
    ]

In [2]:
def list_options_for_dash(df_series):
    options = []
    for i in df_series:
        town = {'label':i, 'value':i}
        dict_copy = town.copy()
        print(town)
        options_df = options.append(dict_copy)
        print(options_df)
    return options

In [92]:
towns = lists_town_home_price_occ_guest_nc['town'].unique()
towns_for_dash = list_options_for_dash(towns)

{'label': 'arapahoe', 'value': 'arapahoe'}
None
{'label': 'atlantic', 'value': 'atlantic'}
None
{'label': 'atlantic beach', 'value': 'atlantic beach'}
None
{'label': 'bald head island', 'value': 'bald head island'}
None
{'label': 'bath', 'value': 'bath'}
None
{'label': 'beaufort', 'value': 'beaufort'}
None
{'label': 'belmont', 'value': 'belmont'}
None
{'label': 'bethel', 'value': 'bethel'}
None
{'label': 'beulaville', 'value': 'beulaville'}
None
{'label': 'bolton', 'value': 'bolton'}
None
{'label': 'burgaw', 'value': 'burgaw'}
None
{'label': 'calabash', 'value': 'calabash'}
None
{'label': 'canton', 'value': 'canton'}
None
{'label': 'carolina beach', 'value': 'carolina beach'}
None
{'label': 'caswell beach', 'value': 'caswell beach'}
None
{'label': 'cedar point', 'value': 'cedar point'}
None
{'label': 'claremont', 'value': 'claremont'}
None
{'label': 'concord', 'value': 'concord'}
None
{'label': 'conway', 'value': 'conway'}
None
{'label': 'davis', 'value': 'davis'}
None
{'label': 'dover

In [77]:
options = [
    {'label':'Google', 'value':'GOOG' },
    {'label': 'Apple', 'value':'AAPL'},
    {'label': 'Amazon', 'value':'AMZN'},
    ]

In [82]:
print(towns)

['arapahoe' 'atlantic' 'atlantic beach' 'bald head island' 'bath'
 'beaufort' 'belmont' 'bethel' 'beulaville' 'bolton' 'burgaw' 'calabash'
 'canton' 'carolina beach' 'caswell beach' 'cedar point' 'claremont'
 'concord' 'conway' 'davis' 'dover' 'eden' 'elizabethtown' 'emerald isle'
 'enfield' 'fair bluff' 'fairfield' 'fayetteville' 'franklin' 'gloucester'
 'greensboro' 'hampstead' 'harkers island' 'harrells' 'havelock'
 'holden beach' 'holly ridge' 'hope mills' 'indian beach' 'jackson'
 'jacksonville' 'jefferson' 'kure beach' 'leland' 'littleton' 'lowell'
 'lumber bridge' 'lumberton' 'madison' 'marshallberg' 'middlesex' 'milton'
 'minnesott beach' 'morehead city' 'mount holly' 'new london' 'newport'
 'north topsail beach' 'oak island' 'ocean isle beach' 'oriental'
 'ossipee' 'pine knoll shores' 'plymouth' 'rocky point' 'roseboro'
 'salisbury' 'sanford' 'shallotte' 'sneads ferry' 'southport'
 'sunset beach' 'surf city' 'swansboro' 'tabor city' 'troy' 'wallace'
 'washington' 'wentworth' '

In [93]:
num_lists_pivot

guest_no,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0
town,,,,,,,,,,,,,,,,
arapahoe,NaN,5.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
atlantic,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
atlantic beach,NaN,13.0,13.0,114.0,16.0,190.0,NaN,76.0,6.0,24.0,NaN,13.0,NaN,NaN,NaN,NaN
bald head island,NaN,NaN,NaN,NaN,NaN,25.0,NaN,45.0,4.0,18.0,3.0,NaN,NaN,NaN,NaN,NaN
bath,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wentworth,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
whiteville,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
wilmington,12.0,307.0,56.0,274.0,54.0,215.0,36.0,79.0,NaN,28.0,NaN,10.0,NaN,NaN,NaN,NaN


In [ ]:
# combined_data df in table_generation notebook
region_proptype_roi = combined_data.groupby(['Region','property_type'])['median_ROI'].median().reset_index()

from plotly.subplots import make_subplots
import plotly.graph_objects as go

neweng = region_proptype_roi[region_proptype_roi['Region'] == 'New England']
carolinas = region_proptype_roi[region_proptype_roi['Region'] == 'Carolinas']

roi_region_subplots = make_subplots(rows=1, cols=2)

roi_region_subplots.append_trace(
    go.Bar(x=neweng['property_type'],
    y=neweng['median_ROI'], name="New England"
), row=1, col=1)

roi_region_subplots.append_trace(go.Bar(
    x=carolinas['property_type'],
    y=carolinas['median_ROI'], name="Carolinas"
), row=1, col=2)

# Change the bar mode
roi_region_subplots.update_layout(title_text='ROI by Property Type', barmode='group')
roi_region_subplots.show()

In [ ]:
with open("reports/report_draft.html",'w') as f:
    f.write(listing_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(price_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(price_v_numlist_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(roi_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(roi_subplots.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(vt_pricing_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(nh_pricing_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(nc_pricing_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(sc_pricing_fig.to_html(full_html=False, include_plotlyjs='cdn'))

In [65]:
listing_data.shape

(5535, 25)

In [59]:
listing_data.lat.isna().value_counts()

False    2775
True     2760
Name: lat, dtype: int64

In [50]:
lat_nas = listing_data[listing_data.lat.isna()]

In [60]:
lat_notnas = listing_data[listing_data.lat.isna() == False]

In [61]:
lat_notnas

,index,id,name,price,displayPrice,monthlyPriceFactor,weeklyPriceFactor,avgRating,reviewsCount,isNewListing,...,beds,baths,title,property_type,town,top_lat,right_long,bottom_lat,left_long,url
0,0,630349775766419403,Pawleys Island Retreat,100.0,$100,1.00,1.00,NaN,0.0,1.0,...,5 beds,3 baths,8 guests,entire_home,Pawleys Island,33.520872,-79.097172,33.421340,-79.290213,https://www.airbnb.com/s/homes?refinement_path...
1,1,574462908054659911,NEW! Pawleys Paradise Family Beach Home w/Golf...,492.0,$492,0.80,0.86,5.00,8.0,0.0,...,8 beds,3.5 baths,14 guests,entire_home,Pawleys Island,33.520872,-79.097172,33.421340,-79.290213,https://www.airbnb.com/s/homes?refinement_path...
2,2,36442397,Peaceful home nestled on golf course,281.0,$281,1.00,0.95,4.64,28.0,0.0,...,5 beds,2 baths,8 guests,entire_home,Pawleys Island,33.520872,-79.097172,33.421340,-79.290213,https://www.airbnb.com/s/homes?refinement_path...
3,3,52108693,"Golf, Beach and Pool! Open 3Bd 2Bath in True Blue",185.0,$185,0.80,0.90,4.92,13.0,0.0,...,6 beds,2 baths,7 guests,entire_home,Pawleys Island,33.520872,-79.097172,33.421340,-79.290213,https://www.airbnb.com/s/homes?refinement_path...
4,4,49887128,Hampton,164.0,$164,1.00,1.00,5.00,6.0,0.0,...,1 bed,1 bath,2 guests,entire_home,Pawleys Island,33.520872,-79.097172,33.421340,-79.290213,https://www.airbnb.com/s/homes?refinement_path...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,8,588577981345597145,Stylish Home Away From Home.,166.0,$166,0.95,0.97,3.67,3.0,0.0,...,5 beds,1 bath,8 guests,entire_home,Miami,25.888884,-80.212473,25.858383,-80.248915,https://www.airbnb.com/s/homes?refinement_path...
9,9,595940242771022474,Clean master room with in suite bathroom,85.0,$85,1.00,1.00,NaN,0.0,1.0,...,1 bed,1 private bath,1 guest,private_room,Miami,25.888884,-80.212473,25.858383,-80.248915,https://www.airbnb.com/s/homes?refinement_path...
10,10,48503887,S. Florida Oasis (N. Miami) Unit 2,271.0,$271,0.75,0.90,4.27,15.0,0.0,...,2 beds,2 baths,4 guests,entire_home,Miami,25.888884,-80.212473,25.858383,-80.248915,https://www.airbnb.com/s/homes?refinement_path...
11,11,50302209,NoMi Oasis Unit 4,259.0,$259,1.00,1.00,NaN,0.0,0.0,...,1 bed,1 bath,2 guests,entire_home,Miami,25.888884,-80.212473,25.858383,-80.248915,https://www.airbnb.com/s/homes?refinement_path...
